In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
import anndata as ad
import time

### Orthotable
read in m to n orthotable

In [ ]:
ortho_tab_s0120_nm_path = os.path.join(*["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysisR",
 "results_GIT","s0120_cyno2human_n_to_m_orthologues.csv.gz"])

In [ ]:
pre = "MH39"
drive = 'F'

In [ ]:
df_ortho_tab_s0120_nm = pd.read_csv(ortho_tab_s0120_nm_path, compression='gzip')

In [ ]:
_,_,_,base_anndata_objects_M24 = h.return_local_paths(drive,pre='M24',add_path = False)
adata_cyno = sc.read_h5ad(os.path.join(base_anndata_objects_M24,'M24' + '_' + 'cyno' + '_anno_celltypes_v0.h5ad'))

In [ ]:
_,_,_,base_anndata_objects_H24 = h.return_local_paths(drive,pre='H24',add_path = False)
adata_human = sc.read_h5ad(os.path.join(base_anndata_objects_H24,'H24' + '_' + 'human' + '_anno_celltypes_v0.h5ad'))

In [ ]:
n_top_genes_per_species = 2000
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
sc.pp.highly_variable_genes(adata_cyno,n_top_genes=n_top_genes_per_species,flavor='seurat_v3',inplace=True)

adata_cyno_hv = adata_cyno[:,adata_cyno.var['highly_variable'] ==True]


In [ ]:
adata_cyno_hv_obs = adata_cyno_hv.obs.copy()

In [ ]:
for cyno_gene in adata_cyno_hv.var.index:
    tab = df_ortho_tab_s0120_nm[df_ortho_tab_s0120_nm['seurat_gene'] == cyno_gene]
    gene_idx = adata_cyno_hv.var_names.get_loc(cyno_gene)
    human_gene_names = tab[tab['Human gene name'].notna()]['Human gene name'].values
    for human_gene_name in list(human_gene_names):
        tab_exact = tab[tab['Human gene name'] == human_gene_name]
        new_gene_name = (tab_exact['Human gene name'].values + '--' + tab_exact['seurat_gene'].values)[0] 
        if tab_exact['ortho_1to1_cyno2human'].values[0] == True:
            ind_list = list(adata_cyno_hv.var.index) 
            ind_list[gene_idx] = new_gene_name
            adata_cyno_hv.var.index = ind_list
        else:
            single_gene_anndata = adata_cyno_hv[:, gene_idx].copy()
            single_gene_anndata.var.index = [new_gene_name]
            adata_cyno_hv = ad.concat([adata_cyno_hv,single_gene_anndata],axis =1)


In [ ]:
adata_cyno_hv.obs = adata_cyno_hv.obs.merge(adata_cyno_hv_obs, left_index=True, right_index=True, how='left')
adata_cyno_hv.write(os.path.join(base_anndata_objects,pre + '_cyno_hv_nomenclatureVAE.h5ad'))  

### Human

In [ ]:

sc.pp.highly_variable_genes(adata_human,n_top_genes=n_top_genes_per_species,flavor='seurat_v3',inplace=True)

adata_human_hv = adata_human[:,adata_human.var['highly_variable'] ==True]

In [ ]:
adata_human_hv_obs = adata_human_hv.obs.copy()

In [ ]:
for human_gene in adata_human_hv.var.index:
    tab = df_ortho_tab_s0120_nm[df_ortho_tab_s0120_nm['Human gene name']== human_gene]
    gene_idx = adata_human_hv.var_names.get_loc(human_gene)
    cyno_gene_names = tab[tab['seurat_gene'].notna()]['seurat_gene'].values
    for cyno_gene_name in list(cyno_gene_names):
            tab_exact = tab[tab['seurat_gene'] == cyno_gene_name]
            new_gene_name = (tab_exact['Human gene name'].values + '--' + tab_exact['seurat_gene'].values)[0] 
            if tab_exact['ortho_1to1_human2cyno'].values[0] == True:
                ind_list = list(adata_human_hv.var.index) 
                ind_list[gene_idx] = new_gene_name
                adata_human_hv.var.index = ind_list
            else:
                single_gene_anndata = adata_human_hv[:, gene_idx].copy()
                single_gene_anndata.var.index = [new_gene_name]
                adata_human_hv = ad.concat([adata_human_hv,single_gene_anndata],axis =1)

In [ ]:
adata_human_hv.obs = adata_human_hv.obs.merge(adata_human_hv_obs, left_index=True, right_index=True, how='left')

In [ ]:
adata_human_hv.write(os.path.join(base_anndata_objects,pre + '_human_hv_nomenclatureVAE.h5ad'))  

### Joined

In [ ]:
adatas_hv = [adata_cyno_hv,adata_human_hv]
adata_hv = ad.concat(adatas_hv)
adata_hv.write(os.path.join(base_anndata_objects,pre + '_joined_gene_nomenclatureVAE.h5ad'))

#### Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()